In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3" # only use this/these gpus
from vllm import LLM, SamplingParams


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers.adapters import AdapterConfig

# Load the base model
base_model_name = "facebook/bart-base"  # Replace with your base model name
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load the adapters
adapter_path = "/path/to/your/adapter"  # Replace with your adapter path
model.load_adapter(adapter_path)

# Merge the adapter weights into the model
model.merge_adapter(adapter_path)

# Save the merged model
merged_model_path = "/path/to/save/merged/model"  # Replace with your desired save path
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)


/home/ubuntu/miniconda3/envs/vllm/lib/python3.10/site-packages/transformers/adapters/__init__.py:27: FutureWarning: The `adapter-transformers` package is deprecated and replaced by the `adapters` package. See https://docs.adapterhub.ml/transitioning.html.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/home/ubuntu/miniconda3/envs/vllm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

TypeError: BartForConditionalGeneration.__init__() got an unexpected keyword argument 'adapter_kwargs'

In [1]:
from huggingface_hub import snapshot_download

sql_lora_path = snapshot_download(repo_id='../cense_csv/models/Llama-2-13b-hf-conversion-cense')

from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest

llm = LLM(model="meta-llama/Llama-2-13b-hf", enable_lora=True)

sampling_params = SamplingParams(
    temperature=0,
    max_tokens=256,
    stop=["[/assistant]"]
)

prompts = [
     "[user] Write a SQL query to answer the question based on the table schema.\n\n context: CREATE TABLE table_name_74 (icao VARCHAR, airport VARCHAR)\n\n question: Name the ICAO for lilongwe international airport [/user] [assistant]",
     "[user] Write a SQL query to answer the question based on the table schema.\n\n context: CREATE TABLE table_name_11 (nationality VARCHAR, elector VARCHAR)\n\n question: When Anchero Pantaleone was the elector what is under nationality? [/user] [assistant]",
]

outputs = llm.generate(
    prompts,
    sampling_params,
    lora_request=LoRARequest("sql_adapter", 1, sql_lora_path)
)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '../cense_csv/models/Llama-2-13b-hf-conversion-cense'. Use `repo_type` argument if needed.

In [ ]:
from vllm import LLM, SamplingParams 
from vllm.model_executor.adapters import lora # need to install forked vllm, on the repo here, before that need to update cuda to 12.1

# Create an LLM.
llm = LLM(model="meta-llama/Llama-2-13b-hf", gpu_memory_utilization=0.05)

# Add LoRA adapter
lora.LoRAModel.from_pretrained(llm.llm_engine.workers[0].model, '../cense_csv/models/Llama-2-13b-hf-conversion-cense')

prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = SamplingParams(temperature=0, top_k=-1)

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [2]:
sampling_params = SamplingParams(temperature=0)#, top_p=0.8)


llm = LLM(
    model = 'meta-llama/Llama-2-13b-hf',
    enable_lora= True
    )# enforce_eager=True)

INFO 07-02 14:47:03 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='meta-llama/Meta-Llama-3-8B', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 

In [3]:


sampling_params = SamplingParams(temperature=0)#, top_p=0.8)
#llm = LLM(
#    model = '../cense_csv/models/Llama-2-13b-hf-conversion-cense',
    #model="meta-llama/Llama-2-7b-hf",
 #   kv_cache_dtype="fp8",
 #   quantization_param_path="./tests/fp8_kv/llama2-7b-fp8-kv/kv_cache_scales.json")


llm = LLM(
    model = '../cense_csv/models/Llama-2-13b-hf-conversion-cense',
    quantization="bitsandbytes", 
    load_format = 'bitsandbytes',
    )# enforce_eager=True)

WARNING 07-02 14:19:56 config.py:217] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 07-02 14:19:56 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='../cense_csv/models/Llama-2-13b-hf-conversion-cense', speculative_config=None, tokenizer='../cense_csv/models/Llama-2-13b-hf-conversion-cense', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=../cense_csv/models/Llama-2-13b-hf-conversion-cense)
INFO 07-02 14:19:57 loader.py:744] Loading we

KeyError: 'base_model.model.model.layers.0.mlp.down_proj.lora_A.qweight'

In [ ]:
prompt = "London is the capital of"
out = llm.generate(prompt, sampling_params)[0].outputs[0].text
print(out)

NameError: name 'llm' is not defined